# Simple autoencoder

Resource comes from https://github.com/L1aoXingyu/pytorch-beginner/tree/master/08-AutoEncoder
Particularly the simple_autoencoder.py file


In [1]:
import os

import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image

In [2]:
if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')

In [3]:
# Q: What is the use of this function?
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

In [4]:
num_epochs = 100
batch_size = 128
learning_rate = 1e-3

In [5]:
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

In [6]:
dataset = MNIST('./data', transform=img_transform, download = True)

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!





In [7]:
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [8]:
# AE model
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        # sw: note the use of nn.Sequential(). It simplifies the syntax a lot.
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True), nn.Linear(64, 12), nn.ReLU(True), nn.Linear(12, 3))
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True), nn.Linear(128, 28 * 28), nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [12]:
# model = autoencoder().cuda()
model = autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

In [14]:
for epoch in range(num_epochs):
#     print(epoch)
    loss_total = 0
    for data in dataloader:
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img).cuda()
        # sw: Variable seems to be used in an older versin of torch. Now it is not necessary.
#         img = Variable(img)
        
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # sw editted the computation of loss function. It might not apply to prediction accuracy.
        loss_total += loss
        
    # ===================log========================
    # sw: learn from the printing format.
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss_total))
#     if epoch % 10 == 0:
#         pic = to_img(output.cpu().data)
#         save_image(pic, './mlp_img/image_{}.png'.format(epoch))
# torch.save(model.state_dict(), './sim_autoencoder.pth')

epoch [1/100], loss:71.6048
epoch [2/100], loss:70.2164
epoch [3/100], loss:69.1126
epoch [4/100], loss:68.1554
epoch [5/100], loss:67.2491
epoch [6/100], loss:66.5621
epoch [7/100], loss:65.9283
epoch [8/100], loss:65.4455
epoch [9/100], loss:64.8500


KeyboardInterrupt: 